In [1]:
import sqlite3
import smtplib
from email.mime.text import MIMEText
from tkinter import *
from tkinter import messagebox
from tkcalendar import DateEntry


conn = sqlite3.connect("hatirlatmalar.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS hatirlatmalar (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    tip TEXT,
    tarih TEXT,
    mesaj TEXT
)
""")
conn.commit()


master = Tk()
master.title("Hatırlatma Uygulaması (Tüm Özellikler)")
master.geometry("900x550")


Label(master, text="Tip:", font="verdana 10").grid(row=0, column=0, padx=10, pady=10, sticky=W)
tip_var = StringVar()
OptionMenu(master, tip_var, "Doğum Günü", "Alışveriş", "Ödeme", "Çalışma", "Genel").grid(row=0, column=1)
tip_var.set("Genel")

Label(master, text="Tarih:", font="verdana 10").grid(row=1, column=0, padx=10, pady=10, sticky=W)
tarih_secici = DateEntry(master, width=12, background="orange", foreground="black", borderwidth=2)
tarih_secici.grid(row=1, column=1)

Label(master, text="Mesaj:", font="verdana 10").grid(row=2, column=0, padx=10, pady=10, sticky=NW)
mesaj_alani = Text(master, height=5, width=40)
mesaj_alani.grid(row=2, column=1, padx=10, pady=10)



def listele():
    liste.delete(0, END)
    cursor.execute("SELECT id, tarih, tip, mesaj FROM hatirlatmalar ORDER BY tarih ASC")
    for row in cursor.fetchall():
        liste.insert(END, f"{row[0]} | {row[1]} | {row[2]}: {row[3]}")

def kaydet():
    tip = tip_var.get()
    tarih = tarih_secici.get()
    mesaj = mesaj_alani.get("1.0", END).strip()

    if not mesaj:
        messagebox.showwarning("Uyarı", "Mesaj alanı boş bırakılamaz.")
        return

    cursor.execute("INSERT INTO hatirlatmalar (tip, tarih, mesaj) VALUES (?, ?, ?)", (tip, tarih, mesaj))
    conn.commit()
    messagebox.showinfo("Başarılı", "Hatırlatma kaydedildi.")
    mesaj_alani.delete("1.0", END)
    listele()

def sil():
    secilen = liste.curselection()
    if not secilen:
        return
    secili_id = liste.get(secilen[0]).split("|")[0].strip()
    cursor.execute("DELETE FROM hatirlatmalar WHERE id = ?", (secili_id,))
    conn.commit()
    listele()

def guncelle():
    secilen = liste.curselection()
    if not secilen:
        return
    secili_id = liste.get(secilen[0]).split("|")[0].strip()
    tip = tip_var.get()
    tarih = tarih_secici.get()
    mesaj = mesaj_alani.get("1.0", END).strip()

    cursor.execute("UPDATE hatirlatmalar SET tip=?, tarih=?, mesaj=? WHERE id=?", (tip, tarih, mesaj, secili_id))
    conn.commit()
    messagebox.showinfo("Güncellendi", "Hatırlatma güncellendi.")
    listele()

def filtrele():
    filtre_tarih = tarih_secici.get()
    liste.delete(0, END)
    cursor.execute("SELECT id, tarih, tip, mesaj FROM hatirlatmalar WHERE tarih=?", (filtre_tarih,))
    for row in cursor.fetchall():
        liste.insert(END, f"{row[0]} | {row[1]} | {row[2]}: {row[3]}")

def gonder_eposta():
    secilen = liste.curselection()
    if not secilen:
        return
    satir = liste.get(secilen[0])
    icerik = "\n".join(satir.split("|")[1:]).strip()

    try:
        alici = "ornek@gmail.com"  # <-- buraya kendi e-posta adresini yaz
        gonderici = "seningmail@gmail.com"
        sifre = "uygulama_sifresi"  # Gmail ayarlarından alınan uygulama şifresi

        msg = MIMEText(icerik)
        msg["Subject"] = "Hatırlatma"
        msg["From"] = gonderici
        msg["To"] = alici

        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(gonderici, sifre)
        server.sendmail(gonderici, alici, msg.as_string())
        server.quit()
        messagebox.showinfo("Gönderildi", "E-posta gönderildi.")
    except Exception as e:
        messagebox.showerror("Hata", f"E-posta gönderilemedi.\n{e}")

def satir_sec(event):
    secilen = liste.curselection()
    if not secilen:
        return
    secili = liste.get(secilen[0]).split("|")
    if len(secili) >= 4:
        tarih_secici.set_date(secili[1].strip())
        tip_var.set(secili[2].strip())
        mesaj_alani.delete("1.0", END)
        mesaj_alani.insert(END, secili[3].strip())


liste = Listbox(master, width=60, height=20)
liste.grid(row=0, column=3, rowspan=8, padx=10, pady=10)
liste.bind("<<ListboxSelect>>", satir_sec)


Button(master, text="Kaydet", command=kaydet, bg="lightgreen", font="verdana 10").grid(row=3, column=1, pady=5)
Button(master, text="Sil", command=sil, bg="salmon", font="verdana 10").grid(row=4, column=1, pady=5)
Button(master, text="Güncelle", command=guncelle, bg="khaki", font="verdana 10").grid(row=5, column=1, pady=5)
Button(master, text="Tarihe Göre Filtrele", command=filtrele, bg="lightblue", font="verdana 10").grid(row=6, column=1, pady=5)
Button(master, text="Tümünü Göster", command=listele, bg="gray90", font="verdana 10").grid(row=7, column=1, pady=5)
Button(master, text="E-posta Gönder", command=gonder_eposta, bg="plum", font="verdana 10").grid(row=8, column=1, pady=5)


listele()
master.mainloop()
